## VARS DwC conversion - larger data set (all species observed in 2001)

Note that initially, this code was run to convert data from 2001. Since then, it has been run on other years (2010, 2017, 1989) to expand list of species names incorrectly matched to WoRMS that need to be managed.

Resources:
- https://dwc.tdwg.org/terms/
- https://tools.gbif.org/dwca-validator/extension.do?id=dwc:Occurrence
- https://www.mbari.org/products/research-software/video-annotation-and-reference-system-vars/query-interface/advanced-user-guide/
- https://www.gbif.org/data-quality-requirements-occurrences

In [1]:
## Imports

import pandas as pd
import numpy as np

import re # for extracting logon info from text file

import jaydebeapi # for connecting to VARS db
import VARS # for connecting to VARS db

from datetime import datetime # for handling dates
import pytz # for handling time zones

import urllib.request, urllib.parse, json # for dealing with WoRMS API and output
import WoRMS # functions for querying WoRMS REST API

### Obtain data from VARS database

In [30]:
# ## Extract logon information from text file

# # Get list of each line in file
# filename = 'VARS_logon_info.txt'
# f = open(filename, 'r')
# lines = f.readlines()
# f.close()

# # Function for extracting information from lines
# def get_single_quoted_text(s):
#     """ 
#     Takes string s and returns any text in s that is between the first set of single quotes, removing whitespace. 
    
#     Example:
#     s = "What if there's more ' than one' sest of single' quotes?"
#     get_single_quoted_text(s) --> 's more'
    
#     """
    
#     extracted_text = re.search('''(?<=')\s*[^']+?\s*(?=')''', s)
#     return(extracted_text.group().strip())

# # Assign logon info
# dr = get_single_quoted_text(lines[2])
# name = get_single_quoted_text(lines[3])
# pw = get_single_quoted_text(lines[4])
# un = get_single_quoted_text(lines[5])
# url = get_single_quoted_text(lines[6])

An explanation of the regex in get_single_quoted_text() can be found here: <br>
https://stackoverflow.com/questions/42002931/regex-extract-string-between-single-quotes-trim-whitespace?rq=1

In [36]:
# ## Build SQL query

# sql = """
#         SELECT index_recorded_timestamp,
#                observation_uuid,
#                concept,
#                observation_group,
#                observer,
#                image_url,
#                depth_meters,
#                latitude,
#                longitude,
#                oxygen_ml_per_l,
#                psi,
#                salinity,
#                temperature_celsius,
#                video_uri,
#                video_sequence_name,
#                chief_scientist
#         FROM annotations a
#         WHERE NOT EXISTS (
#            SELECT DISTINCT observation_uuid
#            FROM annotations b
#            WHERE (
#              (  -- Delete last 2 years of annotations
#              index_recorded_timestamp > DATEADD([year], - 2, GETDATE()) OR
#              index_recorded_timestamp IS NULL OR
#              index_recorded_timestamp < CAST('1970-01-02' AS datetime)
#              )
#            OR ( -- Delete embargoes by dive
#              dive_number IN ('Ventana 50', 'Ventana 217', 'Ventana 218', 'Ventana 248')
#               )
#            OR (
#              dive_number IN ('Tiburon 1001', 'Tiburon 1029', 'Tiburon 1030', 'Tiburon 1031', 'Tiburon 1032', 'Tiburon 1033', 'Tiburon 1034')
#              )
#            OR ( -- Delete embargoes by selectedConcept
#              concept IN (
#                  'Aegina sp. 1',
#                  'Ctenophora',
#                  'Cydippida 2',
#                  'Cydippida',
#                  'Intacta',
#                  'Llyria',
#                  'Lyrocteis',
#                  'Lyroctenidae',
#                  'Mertensia',
#                  'Mertensiidae sp. A',
#                  'Mystery Mollusc',
#                  'Mystery Mollusc',
#                  'Physonectae sp. 1',
#                  'Platyctenida sp. 1',
#                  'Platyctenida',
#                  'Thalassocalycida sp. 1',
#                  'Thalassocalycida',
#                  'Thliptodon sp. A',
#                  'Tjalfiella tristoma',
#                  'Tjalfiella',
#                  'Tjalfiellidae',
#                  'Tuscarantha braueri',
#                  'Tuscarantha luciae',
#                  'Tuscarantha',
#                  'Tuscaretta globosa',
#                  'Tuscaretta',
#                  'Tuscaridium cygneum',
#                  'Tuscaridium',
#                  'Tuscarilla campanella',
#                  'Tuscarilla nationalis',
#                  'Tuscarilla similis',
#                  'Tuscarilla',
#                  'Tuscarora',
#                  'Tuscaroridae'
#                  )
#             )
#         ) AND a.observation_uuid = b.observation_uuid
#     ) AND index_recorded_timestamp >= CAST('1989-01-01' AS datetime) 
#       AND index_recorded_timestamp <= CAST('1989-12-31' AS datetime)
#     """

In [37]:
# ## Query the database

# # Get connection
# conn = VARS.get_db_conn(dr, url, un, pw, name)

# # Submit query
# data = VARS.get_data(conn, sql)

# # Close connection
# conn.close()

In [38]:
# ## Check data is there

# print(data.shape)
# data.head()

(44482, 16)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1989-12-08 18:28:49,B47AB24B-CF05-4611-9762-68D373236C7F,Nanomia bijuga,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
1,1989-12-08 18:29:16,B13CDE3D-C6B3-43CE-A827-4C56C9A0AE1B,Nanomia bijuga,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
2,1989-12-08 18:44:02,5465C311-816E-4C79-B87B-71C5AF30B161,Sebastes,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
3,1989-12-08 18:40:14,2894087F-AD6C-499D-8897-3C831D7A34D8,rock,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
4,1989-12-08 18:33:03,2D19F6CE-E673-4403-8345-CB7200332B94,Eusergestes similis,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter


**Note:** For some reason, this query didn't return any column names. I'll add them here...

In [39]:
# ## Add column names

# data.rename(columns={
#     0:'index_recorded_timestamp',
#     1:'observation_uuid',
#     2:'concept',
#     3:'observation_group',
#     4:'observer',
#     5:'image_url',
#     6:'depth_meters',
#     7:'latitude',
#     8:'longitude',
#     9:'oxygen_ml_per_l',
#     10:'psi',
#     11:'salinity',
#     12:'temperature_celsius',
#     13:'video_uri',
#     14:'video_sequence_name',
#     15:'chief_scientist'
# }, inplace=True)

# data.head()

,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,1989-12-08 18:28:49,B47AB24B-CF05-4611-9762-68D373236C7F,Nanomia bijuga,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
1,1989-12-08 18:29:16,B13CDE3D-C6B3-43CE-A827-4C56C9A0AE1B,Nanomia bijuga,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
2,1989-12-08 18:44:02,5465C311-816E-4C79-B87B-71C5AF30B161,Sebastes,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
3,1989-12-08 18:40:14,2894087F-AD6C-499D-8897-3C831D7A34D8,rock,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
4,1989-12-08 18:33:03,2D19F6CE-E673-4403-8345-CB7200332B94,Eusergestes similis,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter


In [40]:
# ## Save

# data.to_csv('VARS_1989_data.csv', index=False, na_rep='NaN')

### Read in saved data (if not pulled directly from the database)

In [247]:
## Load csv

path = ''
filename = 'VARS_2017_data.csv'
data = pd.read_csv(path+filename, dtype={'image_url': object})

print(data.shape)
data.head()

(309521, 16)


,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,2017-11-13 20:44:35,03DF46F9-F568-45F4-A49D-09208F0BC8C1,Laetmonice,ROV,linda,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,3964.449951,35.131434,-123.001933,2.769,16.299999,34.687000,1.500,urn:tid:mbari.org:D0986-03HD,Doc Ricketts 0986,Ken Smith
1,2017-12-09 16:11:30,769D7A88-A527-4EA0-8AE2-93090E0C67EA,Saccopharynx lavenbergi,ROV,mage,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,1063.329956,36.748602,-122.110794,0.429,55.900002,34.459999,3.793,urn:tid:mbari.org:D0994-02HD,Doc Ricketts 0994,Bruce Robison
2,2017-12-09 16:11:30,769D7A88-A527-4EA0-8AE2-93090E0C67EA,Saccopharynx lavenbergi,ROV,mage,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,1063.329956,36.748602,-122.110794,0.429,55.900002,34.459999,3.793,urn:tid:mbari.org:D0994-02HD,Doc Ricketts 0994,Bruce Robison
3,2017-12-11 00:29:43,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,Galiteuthis,ROV,sbush,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,1207.729980,36.748169,-122.105476,0.676,158.000000,34.507999,3.257,urn:tid:mbari.org:D0995-10HD,Doc Ricketts 0995,Bruce Robison
4,2017-12-11 00:29:43,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,Galiteuthis,ROV,sbush,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,1207.729980,36.748169,-122.105476,0.676,158.000000,34.507999,3.257,urn:tid:mbari.org:D0995-10HD,Doc Ricketts 0995,Bruce Robison


### Pre-processing

In [248]:
## Drop duplicate rows that arise from associations, which we don't care about here

data = data.drop_duplicates()
print(data.shape)
data.head()

(287805, 16)


,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,2017-11-13 20:44:35,03DF46F9-F568-45F4-A49D-09208F0BC8C1,Laetmonice,ROV,linda,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,3964.449951,35.131434,-123.001933,2.769,16.299999,34.687000,1.500,urn:tid:mbari.org:D0986-03HD,Doc Ricketts 0986,Ken Smith
1,2017-12-09 16:11:30,769D7A88-A527-4EA0-8AE2-93090E0C67EA,Saccopharynx lavenbergi,ROV,mage,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,1063.329956,36.748602,-122.110794,0.429,55.900002,34.459999,3.793,urn:tid:mbari.org:D0994-02HD,Doc Ricketts 0994,Bruce Robison
3,2017-12-11 00:29:43,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,Galiteuthis,ROV,sbush,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,1207.729980,36.748169,-122.105476,0.676,158.000000,34.507999,3.257,urn:tid:mbari.org:D0995-10HD,Doc Ricketts 0995,Bruce Robison
5,2017-07-27 17:21:08,82EEDADE-85EB-41EE-A9E4-2ADFA0314334,Engraulis mordax,ROV,lonny,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,169.270004,36.788923,-121.860306,2.531,167.000000,33.859001,9.798,urn:tid:mbari.org:V4054-02HD,Ventana 4054,Chad Waluk
6,2017-06-11 01:04:38,7F79F91C-7F62-4E00-8631-8350F2AF3612,Apolemia,ROV,smartini,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,325.510010,35.498297,-123.999301,1.392,314.100006,34.071999,6.817,urn:tid:mbari.org:D0961-11HD,Doc Ricketts 0961,Steve Haddock


### Convert

In [249]:
## Start with basic event data and change headings

converted = data[['index_recorded_timestamp', 'video_sequence_name', 'observation_group', 'chief_scientist']]
converted = converted.rename(columns={
    'index_recorded_timestamp':'eventDate',
    'video_sequence_name':'eventID',
    'observation_group':'samplingProtocol',
    'chief_scientist':'recordedBy'
})
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy
0,2017-11-13 20:44:35,Doc Ricketts 0986,ROV,Ken Smith
1,2017-12-09 16:11:30,Doc Ricketts 0994,ROV,Bruce Robison
3,2017-12-11 00:29:43,Doc Ricketts 0995,ROV,Bruce Robison
5,2017-07-27 17:21:08,Ventana 4054,ROV,Chad Waluk
6,2017-06-11 01:04:38,Doc Ricketts 0961,ROV,Steve Haddock


In [250]:
## Remove whitespace from eventID

converted['eventID'] = [event.replace(' ', '_') for event in converted['eventID']]
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy
0,2017-11-13 20:44:35,Doc_Ricketts_0986,ROV,Ken Smith
1,2017-12-09 16:11:30,Doc_Ricketts_0994,ROV,Bruce Robison
3,2017-12-11 00:29:43,Doc_Ricketts_0995,ROV,Bruce Robison
5,2017-07-27 17:21:08,Ventana_4054,ROV,Chad Waluk
6,2017-06-11 01:04:38,Doc_Ricketts_0961,ROV,Steve Haddock


**Note** that this code also places an underscore between 'Doc' and 'Ricketts'. It's possible that using 'DocRicketts' could be preferable.

In [251]:
## Add institutionCode

converted['institutionCode'] = 'MBARI'
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode
0,2017-11-13 20:44:35,Doc_Ricketts_0986,ROV,Ken Smith,MBARI
1,2017-12-09 16:11:30,Doc_Ricketts_0994,ROV,Bruce Robison,MBARI
3,2017-12-11 00:29:43,Doc_Ricketts_0995,ROV,Bruce Robison,MBARI
5,2017-07-27 17:21:08,Ventana_4054,ROV,Chad Waluk,MBARI
6,2017-06-11 01:04:38,Doc_Ricketts_0961,ROV,Steve Haddock,MBARI


In [252]:
## Format eventDate

formatted = []

for dt in converted['eventDate']:
    
    # Convert string to datetime
    try:
        dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S.%f') # some datetimes have milliseconds
    except ValueError:
        dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
        
    # Assign UTC timezone
    utc = pytz.UTC
    dt = dt.astimezone(utc)
    
    # Put in ISO format string
    dt = dt.isoformat()
    
    # Save in list
    formatted.append(dt)

converted['eventDate'] = formatted
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode
0,2017-11-14T04:44:35+00:00,Doc_Ricketts_0986,ROV,Ken Smith,MBARI
1,2017-12-10T00:11:30+00:00,Doc_Ricketts_0994,ROV,Bruce Robison,MBARI
3,2017-12-11T08:29:43+00:00,Doc_Ricketts_0995,ROV,Bruce Robison,MBARI
5,2017-07-28T00:21:08+00:00,Ventana_4054,ROV,Chad Waluk,MBARI
6,2017-06-11T08:04:38+00:00,Doc_Ricketts_0961,ROV,Steve Haddock,MBARI


In [253]:
## Add in occurrence-related columns from data, renaming as needed

converted['occurrenceID'] = data['observation_uuid']
converted['scientificName'] = data['concept']
converted['identifiedBy'] = data['observer']
converted['verbatimDepth'] = round(data['depth_meters'], 1)
converted['decimalLatitude'] = data['latitude']
converted['decimalLongitude'] = data['longitude']
converted['dissolvedOxygenInMLPerL'] = data['oxygen_ml_per_l']
converted['pressureInPsi'] = data['psi']
converted['salinity'] = data['salinity']
converted['temperatureInCelsius'] = data['temperature_celsius']
converted['image_url'] = data['image_url']
converted['video_uri'] = data['video_uri']
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,verbatimDepth,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri
0,2017-11-14T04:44:35+00:00,Doc_Ricketts_0986,ROV,Ken Smith,MBARI,03DF46F9-F568-45F4-A49D-09208F0BC8C1,Laetmonice,linda,3964.4,35.131434,-123.001933,2.769,16.299999,34.687000,1.500,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0986-03HD
1,2017-12-10T00:11:30+00:00,Doc_Ricketts_0994,ROV,Bruce Robison,MBARI,769D7A88-A527-4EA0-8AE2-93090E0C67EA,Saccopharynx lavenbergi,mage,1063.3,36.748602,-122.110794,0.429,55.900002,34.459999,3.793,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0994-02HD
3,2017-12-11T08:29:43+00:00,Doc_Ricketts_0995,ROV,Bruce Robison,MBARI,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,Galiteuthis,sbush,1207.7,36.748169,-122.105476,0.676,158.000000,34.507999,3.257,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0995-10HD
5,2017-07-28T00:21:08+00:00,Ventana_4054,ROV,Chad Waluk,MBARI,82EEDADE-85EB-41EE-A9E4-2ADFA0314334,Engraulis mordax,lonny,169.3,36.788923,-121.860306,2.531,167.000000,33.859001,9.798,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V4054-02HD
6,2017-06-11T08:04:38+00:00,Doc_Ricketts_0961,ROV,Steve Haddock,MBARI,7F79F91C-7F62-4E00-8631-8350F2AF3612,Apolemia,smartini,325.5,35.498297,-123.999301,1.392,314.100006,34.071999,6.817,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0961-11HD


In [254]:
## Add coordinateUncertaintyInMeters 

converted['coordinateUncertaintyInMeters'] = 300
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,verbatimDepth,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters
0,2017-11-14T04:44:35+00:00,Doc_Ricketts_0986,ROV,Ken Smith,MBARI,03DF46F9-F568-45F4-A49D-09208F0BC8C1,Laetmonice,linda,3964.4,35.131434,-123.001933,2.769,16.299999,34.687000,1.500,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0986-03HD,300
1,2017-12-10T00:11:30+00:00,Doc_Ricketts_0994,ROV,Bruce Robison,MBARI,769D7A88-A527-4EA0-8AE2-93090E0C67EA,Saccopharynx lavenbergi,mage,1063.3,36.748602,-122.110794,0.429,55.900002,34.459999,3.793,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0994-02HD,300
3,2017-12-11T08:29:43+00:00,Doc_Ricketts_0995,ROV,Bruce Robison,MBARI,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,Galiteuthis,sbush,1207.7,36.748169,-122.105476,0.676,158.000000,34.507999,3.257,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0995-10HD,300
5,2017-07-28T00:21:08+00:00,Ventana_4054,ROV,Chad Waluk,MBARI,82EEDADE-85EB-41EE-A9E4-2ADFA0314334,Engraulis mordax,lonny,169.3,36.788923,-121.860306,2.531,167.000000,33.859001,9.798,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V4054-02HD,300
6,2017-06-11T08:04:38+00:00,Doc_Ricketts_0961,ROV,Steve Haddock,MBARI,7F79F91C-7F62-4E00-8631-8350F2AF3612,Apolemia,smartini,325.5,35.498297,-123.999301,1.392,314.100006,34.071999,6.817,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0961-11HD,300


**Note** that in the 2010 data, there are 22 records where the scientificName is recorded as 'None.' I don't know how these records came to be, but there are few enough of them that I feel comfortable dropping them.

In [258]:
## Drop records where the species name is 'None'

print(converted.shape)
converted = converted[converted['scientificName'].notna()]
print(converted.shape)

(287314, 18)
(287272, 18)


There are also ~400 rows where salinity=0. That doesn't make sense, and I'm assuming salinity is unavailable for these records. Changing them to NaN.

In [259]:
## Change salinity = 0 to NaN

converted[converted['salinity'] == 0] = np.nan

In [260]:
## Get a list of unique species names

converted['scientificName'] = [name.lower().strip() for name in converted['scientificName']]
names = converted['scientificName'].unique()

In [261]:
## Create list of concept names that should not be in species list using WoRMS logs from original runs

def get_concept_names(filename):
    """ Takes a WoRMS log .txt file filename and returns a list of concept names that did not match in WoRMS"""
    
    # Extract data from file
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()

    # Initialize list to hold non-organism concept names
    concepts = []

    # Fill list
    for line in lines:
        if line.split('  ')[0] == 'Url didn\'t work, check name:':
            concepts.append(line.split('  ')[1].strip())
        else:
            concepts.append(line.split('  ')[0].split('for ')[1].split(' checking')[0])           
                
    return concepts

concepts_1989 = get_concept_names('VARS_1989_WoRMS_log.txt')
concepts_2001 = get_concept_names('VARS_2001_WoRMS_log.txt')
concepts_2010 = get_concept_names('VARS_2010_WoRMS_log.txt')
concepts_2017 = get_concept_names('VARS_2017_WoRMS_log.txt')

# Merge these lists into a single list with no duplicate terms
concepts_1989 = list(dict.fromkeys(concepts_1989))
nonorganism_concepts = concepts_1989 + list(set(concepts_2001) - set(concepts_1989))
nonorganism_concepts = nonorganism_concepts + list(set(concepts_2010) - set(nonorganism_concepts))
nonorganism_concepts = nonorganism_concepts + list(set(concepts_2017) - set(nonorganism_concepts))
nonorganism_concepts = list(dict.fromkeys(nonorganism_concepts))

# Remove known biological terms that should have matched in WoRMS but didn't
concepts_to_remove = ['larvacean',
                      'teuthoidea',
                      'phyllospadix-zostera',
                      'gastrozooid',
                      'opisthoteuthis cf. californiana',
                      'retimohnia sp. a',
                      'hydrolagus cf. trolli',
                      'beringraja rhina',
                      'synallactidae gen. et sp. indet.',
                      'tomopterid',
                      'pleuronectiformes 2',
                      'sergestid',
                      'neptunea-buccinum',
                      'pyrosomida',
                      'sebastomus complex',
                      'luciobrotula sp. a',
                      'vitreosalpa',
                      'cirrothauma sp. 1',
                      'nuculana sp. a',
                      'ceriantharia sp. 1',
                      'scotoplanes sp. a',
                      'liparidae sp. 1',
                      'actiniidae sp. 1',
                      'opisthoteuthis sp. a',
                      'coryphaenoides armatus-yaquinae complex',
                      'lycenchelys sp. 1',
                      'bathyraja cf. microtrachys',
                      'vitreosalpa gemini',
                      'coryphaenoides armatus-leptolepis-yaquinae complex',
                      'gastropoda sp. 2',
                      'euphausia 2',
                      'funiculina-halipteris complex',
                      'levinsenia sp. a',
                      'peniagone sp. a',
                      'lrj complex',
                      'doliolinetta',
                      'grimpoteuthis sp. 5',
                      'hydromedusae',
                      'funiculina-halipteris',
                      'neptunea-buccinum complex',
                      'coryphaenoides acrolepis-filifer complex',
                      'poeobius 2',
                      'medusae',
                      'lrj',
                      'graneledoninae',
                      'sicyonis sp. 1',
                      'torquaratoridae sp. 2',
                      'sebastomus',
                      'grimpoteuthis sp. 1',
                      'ceriantharia sp. 2',
                      'hormathiidae sp. 1',
                      'ceriantharia sp. 3',
                      'polychelidae 2',
                      'aeolidiidae sp. 1',
                      'farrea truncata complex',
                      'psamminidae sp. 1',
                      'hexactinellida sp. 2',
                      'benthoctopus sp. 1',
                      'brachiopod',
                      'cydippida sp. a',
                      'tromikosoma sp. 1',
                      'tetrorchis sp. a',
                      'torquaratoridaea',
                      'cladorhizidae sp. a',
                      'oloughlinius sp. 2',
                      'torquaratoridaeb sp. 1',
                      'amphipod',
                      'sebastes aleutianus-melanostictus complex',
                      'oloughlinius sp. 1',
                      'paraliparis sp. 1',
                      'torquaratoridaeb',
                      'kophobelemnon sp. 1',
                      'doliolida 2',
                      'torquaratoridaea sp. 1',
                      'ctenophora sp. a',
                      'torquaratoridaeb sp. 2',
                      'sebastes crameri-melanostomus complex',
                      'asbestopluma monticola',
                      'porifera sp. 1',
                      'psamminidae sp. 2',
                      'asteroidea sp. 1',
                      'pannychia sp. 1',
                      'antedonoidea sp. 1',
                      'benthodytes aff. typica',
                      'octacnemus sp. 1',
                      'enteropneust',
                      'porifera sp. 4',
                      'demospongiae sp. 1',
                      'candelabridae sp. 1',
                      'holothuroidea sp. 8',
                      'hexactinellida sp. 4',
                      'hexactinellida sp. 3', 
                      'asteroidea sp. 2',
                      'nanomia 2',
                      'oneirophanta mutabilis complex',
                      'holothuroidea sp. 4',
                      'mertensiidae sp. b',
                      'pennatulacea sp. 1',
                      'mycale sp. 1',
                      'protoptilum sp. 1',
                      'macellicephala sp. a',
                      'elpidia sp. a',
                      'lyssacinosida sp. 1',
                      'peniagone vitrea- sp. 1 complex',
                      'hexactinellida sp. 6',
                      'octacnemidae sp. 1',
                      'munida sp. a',
                      'doliolenetta',
                      'chondrocladia sp. a',
                      'peniagone sp. 2',
                      'neolithodes diomedae',
                      'acanthascinae sp. 3',
                      'asteroid',
                      'culeolus barryi',
                      'peniagone sp. 1',
                      'camptoplites sp. a',
                      'mycale sp. 2',
                      'fecampiid',
                      'umbellula sp. 1',
                      'forskalia edwardsi',
                      'holothuroidea sp. 6',
                      'acanthascinae sp. 1-4 complex',
                      'situla sp. 1',
                      'hexactinellida sp. 5',
                      'bathydorus sp. 1',
                      'culeolus sp. 2',
                      'prayidae sp. 1',
                      'hyalonema sp. 1',
                      'buccinidae sp. 1',
                      'zoantharia sp. 2',
                      'hexactinellida sp. 1',
                      'zoantharia sp. 1',
                      'acanthascinae sp. 2',
                      'pennatulacea sp. 3'
                     ]
nonorganism_concepts = [concept for concept in nonorganism_concepts if concept not in concepts_to_remove] 

# Add any terms that slipped through
nonorganism_concepts.extend(['iris', 'gastrozooid'])

In [262]:
## Remove nonorganism concepts from names

names = pd.Series(names)
names = names[~names.isin(nonorganism_concepts)]

In [263]:
%%capture cap --no-stderr --no-display

## Look up names in WoRMS and save matched name, name ID and taxon ID to dicts ----- TAKES ~ 15 MINUTES TO DO THE ENTIRE NAMES LIST

name_id_dic, name_dic, id_dic = WoRMS.run_get_worms_from_scientific_name(names, verbose_flag=True)

In [264]:
## Write failed names to log file

with open('VARS_2017_WoRMS_log_20200604.txt', 'w') as f:
    f.write(cap.stdout)

**Terms that seem like they should have matched but didn't from 2001, 2010, 2017, 1989:**
- hydromedusae (**Nancy identified the lowest taxononimc level as Leptolida. WoRMS does not accept this; the accepted name is Hydroidolina.**)
- phyllospadix-zostera (**Phyllospadix OR zostera**)
- gastrozooid (**This is an organism body part.**)
- clam (bivalvia)
- teuthoidea (**This is an old name. Teuthida**)
- medusae (**An MBARI term for a jelly that cannot be identified, even to class level. Phylum Cnidaria.**)
- larvacean house didn't match, but neither did larvacean (**Larvacea?**)
- neptunea-buccinum complex (**Can look complexes up on the MBARI Deep Sea Guide: http://m3.shore.mbari.org/dsg/view/concept/Neptunea-Buccinum%20Complex; family Buccinidae**)
- doliolinetta (**WoRMS will match Dolioletta and Doliolina. These are separate genuses. Doliolina is the same as Doliolinetta.**)
- tomopterid, tomopterid eggcase (**WoRMS will match Tomopteridae, is that correct?**)
- vitreosalpa gemini (**This is a special MBARI name for a species that has not been formally described yet. Family salpidae.**)
- sergestid (**WoRMS will match Sergestidae, is that correct?**)
- sebastomus complex (**Sebastes (also sebastomus) is a subgenus of Sebastes. Sebastes(Sebastomus) should match on WoRMS; aphia id = 991848**)
- graneledoninae (**This is an instance where MBARI may be more up to date than WoRMS. Use family Octopodidae.**)
- pyrosomida (**This is an old class name. Should be Pyrosomatida.**)
- funiculina-halipteris complex (**Family Funiculinidae**)
- torquaratoridaeb (**This is a genus in the family Torquaratoridae http://dsg.mbari.org/dsg/images/concept/Enteropneusta. There is also a torquaratoridaeA. Family Torquaratoridae**).
- nectophore (**This is an organism body part.**)
- lrj complex (**Little Red Jellies complex. Family Rhopalonematidae.**)
- brachiopod basket doesn't match, meaning that brachiopod won't
- lophophore 
- amphipod tube mat reduces to amphipod, but doesn't match

Also **note** that some terms that did match weren't necessarily species:
- cylinder (**This should not be matched; it refers to a piece of equipment.**)
- phyllospadix settlement rake matched to phyllospadix
- mars node matched to mars
- bathochordaeus sinker matched to bathochordaeus (**is this appropriate? Does this indicate just the mucus house?**)
- coryphaenoides acrolepis-filifer complex matched to coryphaenoides (**This is fine. Coryphaenoides is a genus of fish, and the species cannot easily be distinguished by eye. Just match the genus.**)
- coryphaenoides armatus-leptolepis-yaquinae complex matched to coryphaenoides
- coryphaenoides armatus-yaquinae complex matched to coryphaenoides
- medusa carcass matched to medusa
- phyllospadix-zostera detritus is reduced to phyllospadix-zostera and fails
- sergestid molt matched to sergestid
- sebastes aleutianus-melanostictus complex matched to sebastes (**This is fine.**)
- sebastes crameri-melanostomus complex matched to sebastes
- polychaeta tube matched to polychaeta
- opisthoteuthis cf. californiana matched to opisthoteuthis (**This is fine. cf. stands for the Latin conferre, meaning that the organism observed had features consistent with opistoteuthis californiana, but the species ID is uncertain.**)
- pyrosoma detritus matched to pyrosoma
- oneirophanta mutabilis complex matched to oneirophanta
- farrea truncata complex matched to farrea (**This is fine.**)

#### If desired, save dictionaries as json

In [265]:
## Save dictionaries

with open('VARS_2017_name_id_dict.json', 'w') as fp:
    json.dump(name_id_dic, fp, sort_keys=True, indent=4)
    
with open('VARS_2017_name_name_dict.json', 'w') as fp:
    json.dump(name_dic, fp, sort_keys=True, indent=4)
    
with open('VARS_2017_name_taxid_dict.json', 'w') as fp:
    json.dump(id_dic, fp, sort_keys=True, indent=4)

#### If desired, read dictionaries rather than querying WoRMS

In [23]:
## Load dictionaries

with open('VARS_2001_name_id_dict.json') as f:
  name_id_dic = json.load(f)

with open('VARS_2001_name_name_dict.json') as f:
  name_dic = json.load(f)

with open('VARS_2001_name_taxid_dict.json') as f:
  id_dic = json.load(f)

#### Handle organisms that should have matched on WoRMS but didn't

In [266]:
## Create a dictionary mapping biological names that didn't match on WoRMS to names that should match

VARS_to_WoRMS_dict = {'hydromedusae':'hydroidolina',
                      'teuthoidea':'teuthida',
                      'lrj complex':'rhopalonematidae',
                      'medusae':'cnidaria',
                      'neptunea-buccinum complex':'buccinidae',
                      'doliolinetta':'doliolina',
                      'doliolenetta':'doliolina',
                      'vitreosalpa gemini':'salpidae',
                      'vitreosalpa':'salpidae',
                      'sebastomus complex':'sebastes (sebastomus)',
                      'graneledoninae':'octopodidae',
                      'funiculina-halipteris complex':'funiculinidae',
                      'torquaratoridaea sp. 1':'torquaratoridae',
                      'torquaratoridaeb sp. 1':'torquaratoridae',
                      'torquaratoridaeb sp. 2':'torquaratoridae',
                      'funiculina-halipteris complex':'funiculinidae'
                     }

In [267]:
## Run these additional terms through WoRMS

revised_concepts = ['hydroidolina', 'teuthida', 'rhopalonematidae', 'cnidaria', 'buccinidae', 'doliolina', 'salpidae', 'sebastes (sebastomus)', 'octopodidae', 'funiculinidae',
                   'torquaratoridae', 'funiculinidae']
revised_name_id_dic, revised_name_dic, revised_id_dic = WoRMS.run_get_worms_from_scientific_name(revised_concepts, verbose_flag=True)

In [268]:
## Add values for revised names to original WoRMS output

name_id_dic.update(revised_name_id_dic)
name_dic.update(revised_name_dic)
id_dic.update(revised_id_dic)

In [269]:
## Create columns from WoRMS data

# Replace names that don't have a WoRMS match in scientificName with revised names
converted['scientificName'].replace(VARS_to_WoRMS_dict, inplace=True)

# Create scientificNameID column with the same content as scientificName - strip to ensure no whitespace, lowercase
converted['scientificNameID'] = converted['scientificName'].str.strip().str.lower()

# Use dictionary to replace scientific names with name IDs
converted.replace({'scientificNameID':name_id_dic}, inplace=True)

# Repeat to create taxonID
converted['taxonID'] = converted['scientificName'].str.strip().str.lower()
converted.replace({'taxonID':id_dic}, inplace=True)

converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,verbatimDepth,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID
0,2017-11-14T04:44:35+00:00,Doc_Ricketts_0986,ROV,Ken Smith,MBARI,03DF46F9-F568-45F4-A49D-09208F0BC8C1,laetmonice,linda,3964.4,35.131434,-123.001933,2.769,16.299999,34.687000,1.500,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0986-03HD,300.0,urn:lsid:marinespecies.org:taxname:129196,129196
1,2017-12-10T00:11:30+00:00,Doc_Ricketts_0994,ROV,Bruce Robison,MBARI,769D7A88-A527-4EA0-8AE2-93090E0C67EA,saccopharynx lavenbergi,mage,1063.3,36.748602,-122.110794,0.429,55.900002,34.459999,3.793,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0994-02HD,300.0,urn:lsid:marinespecies.org:taxname:274336,274336
3,2017-12-11T08:29:43+00:00,Doc_Ricketts_0995,ROV,Bruce Robison,MBARI,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,galiteuthis,sbush,1207.7,36.748169,-122.105476,0.676,158.000000,34.507999,3.257,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0995-10HD,300.0,urn:lsid:marinespecies.org:taxname:137848,137848
5,2017-07-28T00:21:08+00:00,Ventana_4054,ROV,Chad Waluk,MBARI,82EEDADE-85EB-41EE-A9E4-2ADFA0314334,engraulis mordax,lonny,169.3,36.788923,-121.860306,2.531,167.000000,33.859001,9.798,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V4054-02HD,300.0,urn:lsid:marinespecies.org:taxname:272286,272286
6,2017-06-11T08:04:38+00:00,Doc_Ricketts_0961,ROV,Steve Haddock,MBARI,7F79F91C-7F62-4E00-8631-8350F2AF3612,apolemia,smartini,325.5,35.498297,-123.999301,1.392,314.100006,34.071999,6.817,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0961-11HD,300.0,urn:lsid:marinespecies.org:taxname:135393,135393


In [270]:
## Remove rows that didn't have a WoRMS match

converted = converted[converted['scientificName'].str.strip().str.lower() != converted['scientificNameID']]
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,verbatimDepth,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID
0,2017-11-14T04:44:35+00:00,Doc_Ricketts_0986,ROV,Ken Smith,MBARI,03DF46F9-F568-45F4-A49D-09208F0BC8C1,laetmonice,linda,3964.4,35.131434,-123.001933,2.769,16.299999,34.687000,1.500,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0986-03HD,300.0,urn:lsid:marinespecies.org:taxname:129196,129196
1,2017-12-10T00:11:30+00:00,Doc_Ricketts_0994,ROV,Bruce Robison,MBARI,769D7A88-A527-4EA0-8AE2-93090E0C67EA,saccopharynx lavenbergi,mage,1063.3,36.748602,-122.110794,0.429,55.900002,34.459999,3.793,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0994-02HD,300.0,urn:lsid:marinespecies.org:taxname:274336,274336
3,2017-12-11T08:29:43+00:00,Doc_Ricketts_0995,ROV,Bruce Robison,MBARI,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,galiteuthis,sbush,1207.7,36.748169,-122.105476,0.676,158.000000,34.507999,3.257,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0995-10HD,300.0,urn:lsid:marinespecies.org:taxname:137848,137848
5,2017-07-28T00:21:08+00:00,Ventana_4054,ROV,Chad Waluk,MBARI,82EEDADE-85EB-41EE-A9E4-2ADFA0314334,engraulis mordax,lonny,169.3,36.788923,-121.860306,2.531,167.000000,33.859001,9.798,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V4054-02HD,300.0,urn:lsid:marinespecies.org:taxname:272286,272286
6,2017-06-11T08:04:38+00:00,Doc_Ricketts_0961,ROV,Steve Haddock,MBARI,7F79F91C-7F62-4E00-8631-8350F2AF3612,apolemia,smartini,325.5,35.498297,-123.999301,1.392,314.100006,34.071999,6.817,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0961-11HD,300.0,urn:lsid:marinespecies.org:taxname:135393,135393


In [271]:
## Replace scientificName with matched scientific names from WoRMS

converted['scientificName'] = converted['scientificName'].str.strip().str.lower()
converted['scientificName'].replace(name_dic, inplace=True)
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,verbatimDepth,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID
0,2017-11-14T04:44:35+00:00,Doc_Ricketts_0986,ROV,Ken Smith,MBARI,03DF46F9-F568-45F4-A49D-09208F0BC8C1,Laetmonice,linda,3964.4,35.131434,-123.001933,2.769,16.299999,34.687000,1.500,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0986-03HD,300.0,urn:lsid:marinespecies.org:taxname:129196,129196
1,2017-12-10T00:11:30+00:00,Doc_Ricketts_0994,ROV,Bruce Robison,MBARI,769D7A88-A527-4EA0-8AE2-93090E0C67EA,Saccopharynx lavenbergi,mage,1063.3,36.748602,-122.110794,0.429,55.900002,34.459999,3.793,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0994-02HD,300.0,urn:lsid:marinespecies.org:taxname:274336,274336
3,2017-12-11T08:29:43+00:00,Doc_Ricketts_0995,ROV,Bruce Robison,MBARI,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,Galiteuthis,sbush,1207.7,36.748169,-122.105476,0.676,158.000000,34.507999,3.257,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0995-10HD,300.0,urn:lsid:marinespecies.org:taxname:137848,137848
5,2017-07-28T00:21:08+00:00,Ventana_4054,ROV,Chad Waluk,MBARI,82EEDADE-85EB-41EE-A9E4-2ADFA0314334,Engraulis mordax,lonny,169.3,36.788923,-121.860306,2.531,167.000000,33.859001,9.798,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V4054-02HD,300.0,urn:lsid:marinespecies.org:taxname:272286,272286
6,2017-06-11T08:04:38+00:00,Doc_Ricketts_0961,ROV,Steve Haddock,MBARI,7F79F91C-7F62-4E00-8631-8350F2AF3612,Apolemia,smartini,325.5,35.498297,-123.999301,1.392,314.100006,34.071999,6.817,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0961-11HD,300.0,urn:lsid:marinespecies.org:taxname:135393,135393


In [272]:
## Create additional needed columns

converted['nameAccordingTo'] = 'WoRMS'
converted['occurrenceStatus'] = 'present'
converted['basisOfRecord'] = 'MachineObservation'
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,verbatimDepth,decimalLatitude,...,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord
0,2017-11-14T04:44:35+00:00,Doc_Ricketts_0986,ROV,Ken Smith,MBARI,03DF46F9-F568-45F4-A49D-09208F0BC8C1,Laetmonice,linda,3964.4,35.131434,...,34.687000,1.500,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0986-03HD,300.0,urn:lsid:marinespecies.org:taxname:129196,129196,WoRMS,present,MachineObservation
1,2017-12-10T00:11:30+00:00,Doc_Ricketts_0994,ROV,Bruce Robison,MBARI,769D7A88-A527-4EA0-8AE2-93090E0C67EA,Saccopharynx lavenbergi,mage,1063.3,36.748602,...,34.459999,3.793,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0994-02HD,300.0,urn:lsid:marinespecies.org:taxname:274336,274336,WoRMS,present,MachineObservation
3,2017-12-11T08:29:43+00:00,Doc_Ricketts_0995,ROV,Bruce Robison,MBARI,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,Galiteuthis,sbush,1207.7,36.748169,...,34.507999,3.257,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0995-10HD,300.0,urn:lsid:marinespecies.org:taxname:137848,137848,WoRMS,present,MachineObservation
5,2017-07-28T00:21:08+00:00,Ventana_4054,ROV,Chad Waluk,MBARI,82EEDADE-85EB-41EE-A9E4-2ADFA0314334,Engraulis mordax,lonny,169.3,36.788923,...,33.859001,9.798,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V4054-02HD,300.0,urn:lsid:marinespecies.org:taxname:272286,272286,WoRMS,present,MachineObservation
6,2017-06-11T08:04:38+00:00,Doc_Ricketts_0961,ROV,Steve Haddock,MBARI,7F79F91C-7F62-4E00-8631-8350F2AF3612,Apolemia,smartini,325.5,35.498297,...,34.071999,6.817,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0961-11HD,300.0,urn:lsid:marinespecies.org:taxname:135393,135393,WoRMS,present,MachineObservation


In [273]:
## Assemble associatedMedia

associatedMedia = []

for occ_id in converted['occurrenceID'].unique():
    
    # Select data associated with that occurrenceID:
    selected = converted[converted['occurrenceID'] == occ_id]
    
    # Retrieve unique image and video files
    image_files = selected['image_url'].drop_duplicates()
    video_files = selected['video_uri'].drop_duplicates()
    
    # Remove any NaN values
    image_files = image_files.dropna()
    video_files = video_files.dropna()
    
    # Join image and video files
    media = pd.concat([image_files, video_files])
    
    # Create a string with all the urls
    url_str = ''
    for url in media: url_str = url_str + url + ' | '
    url_str = url_str[0:-3]
    
    # Add to associatedMedia
    associatedMedia.append(url_str)

**Note:** That took a little under 15 minutes to run. I don't know if there's a better/easier way to do this?

In [274]:
## Add to df

# First, need to remove rows with duplicate occurrenceIDs
converted = converted.drop_duplicates(subset='occurrenceID', keep="first")

# Add associatedMedia
converted['associatedMedia'] = associatedMedia
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,verbatimDepth,decimalLatitude,...,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord,associatedMedia
0,2017-11-14T04:44:35+00:00,Doc_Ricketts_0986,ROV,Ken Smith,MBARI,03DF46F9-F568-45F4-A49D-09208F0BC8C1,Laetmonice,linda,3964.4,35.131434,...,1.500,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0986-03HD,300.0,urn:lsid:marinespecies.org:taxname:129196,129196,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...
1,2017-12-10T00:11:30+00:00,Doc_Ricketts_0994,ROV,Bruce Robison,MBARI,769D7A88-A527-4EA0-8AE2-93090E0C67EA,Saccopharynx lavenbergi,mage,1063.3,36.748602,...,3.793,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0994-02HD,300.0,urn:lsid:marinespecies.org:taxname:274336,274336,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...
3,2017-12-11T08:29:43+00:00,Doc_Ricketts_0995,ROV,Bruce Robison,MBARI,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,Galiteuthis,sbush,1207.7,36.748169,...,3.257,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0995-10HD,300.0,urn:lsid:marinespecies.org:taxname:137848,137848,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...
5,2017-07-28T00:21:08+00:00,Ventana_4054,ROV,Chad Waluk,MBARI,82EEDADE-85EB-41EE-A9E4-2ADFA0314334,Engraulis mordax,lonny,169.3,36.788923,...,9.798,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V4054-02HD,300.0,urn:lsid:marinespecies.org:taxname:272286,272286,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...
6,2017-06-11T08:04:38+00:00,Doc_Ricketts_0961,ROV,Steve Haddock,MBARI,7F79F91C-7F62-4E00-8631-8350F2AF3612,Apolemia,smartini,325.5,35.498297,...,6.817,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...,urn:tid:mbari.org:D0961-11HD,300.0,urn:lsid:marinespecies.org:taxname:135393,135393,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...


In [275]:
## Drop extra columns

converted = converted.drop(['image_url', 'video_uri'], axis=1)
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,verbatimDepth,decimalLatitude,...,pressureInPsi,salinity,temperatureInCelsius,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord,associatedMedia
0,2017-11-14T04:44:35+00:00,Doc_Ricketts_0986,ROV,Ken Smith,MBARI,03DF46F9-F568-45F4-A49D-09208F0BC8C1,Laetmonice,linda,3964.4,35.131434,...,16.299999,34.687000,1.500,300.0,urn:lsid:marinespecies.org:taxname:129196,129196,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...
1,2017-12-10T00:11:30+00:00,Doc_Ricketts_0994,ROV,Bruce Robison,MBARI,769D7A88-A527-4EA0-8AE2-93090E0C67EA,Saccopharynx lavenbergi,mage,1063.3,36.748602,...,55.900002,34.459999,3.793,300.0,urn:lsid:marinespecies.org:taxname:274336,274336,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...
3,2017-12-11T08:29:43+00:00,Doc_Ricketts_0995,ROV,Bruce Robison,MBARI,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,Galiteuthis,sbush,1207.7,36.748169,...,158.000000,34.507999,3.257,300.0,urn:lsid:marinespecies.org:taxname:137848,137848,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...
5,2017-07-28T00:21:08+00:00,Ventana_4054,ROV,Chad Waluk,MBARI,82EEDADE-85EB-41EE-A9E4-2ADFA0314334,Engraulis mordax,lonny,169.3,36.788923,...,167.000000,33.859001,9.798,300.0,urn:lsid:marinespecies.org:taxname:272286,272286,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...
6,2017-06-11T08:04:38+00:00,Doc_Ricketts_0961,ROV,Steve Haddock,MBARI,7F79F91C-7F62-4E00-8631-8350F2AF3612,Apolemia,smartini,325.5,35.498297,...,314.100006,34.071999,6.817,300.0,urn:lsid:marinespecies.org:taxname:135393,135393,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...


In [276]:
## Reorder columns

converted = converted[['eventID', 'eventDate', 'samplingProtocol', 'recordedBy', 'institutionCode', 'occurrenceID', 'scientificName', 'scientificNameID', 'taxonID', 
                       'nameAccordingTo', 'occurrenceStatus', 'basisOfRecord', 'identifiedBy', 'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters',
                       'verbatimDepth', 'dissolvedOxygenInMLPerL', 'pressureInPsi', 'salinity', 'temperatureInCelsius', 'associatedMedia']]
converted.head()

,eventID,eventDate,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,scientificNameID,taxonID,nameAccordingTo,...,identifiedBy,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,verbatimDepth,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,associatedMedia
0,Doc_Ricketts_0986,2017-11-14T04:44:35+00:00,ROV,Ken Smith,MBARI,03DF46F9-F568-45F4-A49D-09208F0BC8C1,Laetmonice,urn:lsid:marinespecies.org:taxname:129196,129196,WoRMS,...,linda,35.131434,-123.001933,300.0,3964.4,2.769,16.299999,34.687000,1.500,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...
1,Doc_Ricketts_0994,2017-12-10T00:11:30+00:00,ROV,Bruce Robison,MBARI,769D7A88-A527-4EA0-8AE2-93090E0C67EA,Saccopharynx lavenbergi,urn:lsid:marinespecies.org:taxname:274336,274336,WoRMS,...,mage,36.748602,-122.110794,300.0,1063.3,0.429,55.900002,34.459999,3.793,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...
3,Doc_Ricketts_0995,2017-12-11T08:29:43+00:00,ROV,Bruce Robison,MBARI,7F34CA83-73CF-43B4-A0C6-64CAAE1AA3A7,Galiteuthis,urn:lsid:marinespecies.org:taxname:137848,137848,WoRMS,...,sbush,36.748169,-122.105476,300.0,1207.7,0.676,158.000000,34.507999,3.257,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...
5,Ventana_4054,2017-07-28T00:21:08+00:00,ROV,Chad Waluk,MBARI,82EEDADE-85EB-41EE-A9E4-2ADFA0314334,Engraulis mordax,urn:lsid:marinespecies.org:taxname:272286,272286,WoRMS,...,lonny,36.788923,-121.860306,300.0,169.3,2.531,167.000000,33.859001,9.798,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...
6,Doc_Ricketts_0961,2017-06-11T08:04:38+00:00,ROV,Steve Haddock,MBARI,7F79F91C-7F62-4E00-8631-8350F2AF3612,Apolemia,urn:lsid:marinespecies.org:taxname:135393,135393,WoRMS,...,smartini,35.498297,-123.999301,300.0,325.5,1.392,314.100006,34.071999,6.817,http://search.mbari.org/ARCHIVE/frameGrabs/Doc...


In [277]:
## Save

converted.to_csv('VARS_2017_converted.csv', index=False, na_rep='NaN')

**Note:** This pipeline takes 20-25 minutes to run on ~200,000 records. Primarily, this time comes from 1) WoRMS look-up and 2) assembly of associatedMedia.

### Remaining issues

1. coordinateUncertaintyInMeters **Talked with Dave Caress and Jenny Paduan about this. It is an extremely complex issue. The short story is that it's probably possible to calculate the *best* possible error from first principles, and then talk to Linda Kuntz (recently retired) to figure out which dives had known problems and the best possible error shouldn't be applied. When pressed for an estimate of *worst* possible error, it sounds like it varies with depth and tether length, but could be up to 300 m. After consulting with Patrick, it seems like the best approach is to give the upper limit of 300 m for now.**
2. How accurate are depths? Probably not to 6 decimal places. **Talked to Dave Caress and Jenny Paduan about this. The accuracy of pressure sensors is some percentage of their range; for the sensors on MBARI's ROVs and AUVs, their error is in the range of 1-10 cm. Bear in mind, though, that the pressure sensor is on top of the ROV (approximately 2 meters off the bottom), and that tide has not been accounted for. Tidal range in Monterey Bay is a little under 8 ft, or about 2.5 m. In the past, Jenny has rounded depth**
3. Check through WoRMS log for names that should have matched **Got a detailed response to my questions from Nancy Jacobsen-Stout. Have incorporated these changes.**
4. Faster way to assemble associatedMedia?

**Note:** The WoRMS lookup could probably be significantly accelerated by compiling a list of rejected terms from the WoRMS logs I've already created, and removing matching records before querying WoRMS. **Completed - only sped things up a little.**